In [469]:
import pandas as pd
import warnings

from dateutil.parser import parse
warnings.filterwarnings('ignore')

In [470]:
df_as = pd.read_csv("../data/original/ActivitySummary.csv")
df_as.head()

,dateComponents,activeEnergyBurned,activeEnergyBurnedGoal,activeEnergyBurnedUnit,appleExerciseTime,appleExerciseTimeGoal,appleStandHours,appleStandHoursGoal
0,2018-01-14,0.000,0,kcal,0,30,0,12
1,2018-01-15,0.000,0,kcal,0,30,0,12
2,2018-01-16,356.837,630,kcal,20,30,13,12
3,2018-01-17,409.932,630,kcal,21,30,17,12
4,2018-01-18,369.308,630,kcal,10,30,15,12


In [471]:
import datetime

# only relevant features
df_as_filtered = df_as[['dateComponents','activeEnergyBurned','appleExerciseTime','appleStandHours']]
       
# convert to time
for (index, value) in df_as_filtered.iterrows():
    correct_time = datetime.datetime.strptime(value['dateComponents'], '%Y-%m-%d')
    df_as_filtered.at[index, 'creationDate'] = correct_time
    

df_as_final = df_as_filtered[['creationDate', 'activeEnergyBurned', 'appleExerciseTime', 'appleStandHours']]

df_as_final.head()

,creationDate,activeEnergyBurned,appleExerciseTime,appleStandHours
0,2018-01-14,0.000,0,0
1,2018-01-15,0.000,0,0
2,2018-01-16,356.837,20,13
3,2018-01-17,409.932,21,17
4,2018-01-18,369.308,10,15


In [499]:
df_copy = df_as_final.set_index('creationDate')
df_copy.head()

,activeEnergyBurned,appleExerciseTime,appleStandHours
creationDate,,,
2018-01-14,0.000,0,0
2018-01-15,0.000,0,0
2018-01-16,356.837,20,13
2018-01-17,409.932,21,17
2018-01-18,369.308,10,15


In [579]:
df_hrv = pd.read_csv("../data/original/HeartRateVariabilitySDNN.csv")
df_hrv.shape

(932, 9)

In [532]:
df_hrv_filtered = df_hrv[['creationDate', 'value']]

# df_hrv_copy = df_hrv_filtered.set_index('creationDate')
# df_hrv_copy.head()

# convert from string to datetime
df_hrv_filtered['creationDate'] = pd.to_datetime(df_hrv_filtered['creationDate'])
# mean per day
df_hrv_final = df_hrv_filtered.groupby(df_hrv_filtered['creationDate'].dt.date).mean()

df_hrv_filtered.head()



,creationDate,value
0,2018-01-16 09:50:18+02:00,32.3970
1,2018-01-16 09:55:05+02:00,45.1124
2,2018-01-16 14:52:15+02:00,27.3781
3,2018-01-16 21:24:11+02:00,41.4216
4,2018-01-18 09:08:50+02:00,41.0526


In [474]:
df_mindful = pd.read_csv("../data/original/MindfulSession.csv")
df_mindful.head()

,sourceName,sourceVersion,device,type,unit,creationDate,startDate,endDate,value
0,Headspace,8769,NaN,MindfulSession,NaN,2018-01-14 11:16:40 +0200,2018-01-14 11:15:45 +0200,2018-01-14 11:16:05 +0200,NaN
1,Headspace,8769,NaN,MindfulSession,NaN,2018-01-14 11:16:40 +0200,2018-01-14 11:16:06 +0200,2018-01-14 11:16:11 +0200,NaN
2,Headspace,8769,NaN,MindfulSession,NaN,2018-01-14 11:16:40 +0200,2018-01-14 11:16:12 +0200,2018-01-14 11:16:14 +0200,NaN
3,Headspace,8769,NaN,MindfulSession,NaN,2018-01-14 11:16:40 +0200,2018-01-14 11:16:15 +0200,2018-01-14 11:16:18 +0200,NaN
4,Headspace,8769,NaN,MindfulSession,NaN,2018-01-14 11:16:40 +0200,2018-01-14 11:16:20 +0200,2018-01-14 11:16:20 +0200,NaN


In [475]:
# removed redundant features
df_mindful_filtered = df_mindful[['creationDate','startDate', 'endDate']]
df_mindful_filtered.head()

# calculated duration between startDate and endDate
df_mindful_filtered['duration'] = [time for 
                               time in 
                               pd.to_datetime(df_mindful['endDate']) - pd.to_datetime(df_mindful['startDate'])]

df_mindful_final = df_mindful_filtered[['creationDate', 'duration']]

# converted duration to seconds format
df_mindful_filtered['seconds'] = df_mindful_filtered['duration'].dt.total_seconds()
df_mindfulness = df_mindful_filtered[['creationDate', 'seconds']]

# grouped days by seconds meditated
df_final_mindful = df_mindfulness.groupby(pd.to_datetime(df_mindfulness['creationDate']).dt.date).sum()
df_final_mindful.head()



,seconds
creationDate,
2018-01-14,1393.0
2018-01-16,689.0
2018-01-20,1382.0
2018-01-22,1792.0
2018-01-23,705.0


In [476]:
df_step = pd.read_csv("../data/original/StepCount.csv")
df_step.shape

(38408, 9)

In [477]:
df_step_filtered = df_step[['creationDate', 'value']]
df_step_filtered.head()

,creationDate,value
0,2018-01-13 00:37:45 +0200,12
1,2018-01-13 00:37:45 +0200,4
2,2018-01-13 01:39:00 +0200,77
3,2018-01-13 10:38:15 +0200,16
4,2018-01-13 15:42:21 +0200,13


In [581]:
# convert from string to datetime
df_step_filtered['creationDate'] = pd.to_datetime(df_step_filtered['creationDate'])
# sum total per day
df_step_final = df_step_filtered.groupby(df_step_filtered['creationDate'].dt.date).sum()

df_step_final.shape

# for (item, value) in df_step_final.iterrows():
#     print(value)
#     break


(626, 1)

In [494]:
for (index, row) in df_step_final.reset_index().iterrows():
    print(index)
    print(row)
    creationDate, value = row
    print(creationDate)
    print(value)
    break

0
creationDate    2018-01-13
value                 8337
Name: 0, dtype: object
2018-01-13
8337


In [479]:
df_flights_climbed = pd.read_csv("../data/original/FlightsClimbed.csv")
# df_flights_climbed.head()

In [480]:
df_flights_climbed_filtered = df_flights_climbed[['creationDate', 'value']]
# df_flights_climbed_filtered.shape

In [481]:
time = df_flights_climbed_filtered['creationDate']
time = pd.to_datetime(time)

# df_flights_climbed_filtered.groupby(df_flights_climbed_filtered['creationDate'].dt.date).sum()

df_flights_final = df_flights_climbed_filtered.groupby(time.dt.date).sum()
df_flights_final.head()



# df_flights_climbed_filtered.head()
# DID:
# 1 - removed redundant columns
# 2 - sum per day of flights climbed

,value
creationDate,
2018-01-13,3
2018-01-14,2
2018-01-15,23
2018-01-16,9
2018-01-17,5


## Aggregate ActivitySummary with all activities

In [577]:
results = pd.DataFrame()
results['stepCount'] = df_step_final['value']
results['flightsClimbed'] = df_flights_final['value']
results['meditationSeconds'] = df_final_mindful['seconds']
results['activeEnergyBurned'] = df_copy['activeEnergyBurned']
results['appleExerciseTime'] = df_copy['appleExerciseTime']
results['appleStandHours'] = df_copy['appleStandHours']
results['hrv'] = df_hrv_final['value']


results.to_csv('../data/summary_shaped.csv')




In [578]:
hrv_merged = pd.DataFrame()

def at_work(row):
    hour = pd.to_datetime(row['creationDate']).hour
    if(hour >= 9 and hour <17):
        return 1
    return 0
    
# winter is from 1st of November to 1st of April    
def is_winter(row):
    month = pd.to_datetime(row['creationDate'], format='%d/%m').month
    
    if(month >= 10 or month < 4):
        return 1
    return 0
    
hrv_merged['date'] = df_hrv_filtered['creationDate']
hrv_merged['value'] = df_hrv_filtered['value']
work_hours = []
winter_time = []
for (index, row) in df_hrv_filtered.iterrows():
#     print(index)
#     print(row)
#     break
#     print(row, type(row['value']))
    work_hours.append(at_work(row))
    winter_time.append(is_winter(row))
    
hrv_merged['isAtWork'] = work_hours
hrv_merged['isWinter'] = winter_time
hrv_merged.to_csv('../data/hrv_shaped.csv')